# Inventory Optimization with Snowflake ML Jobs

This notebook demonstrates how to run a Pyomo-based inventory optimization as a Snowflake ML Job.

**Prerequisites:**
- `INVENTORY_OPTIMIZATION.py` uploaded to stage `ML_JOBS_STAGE`
- Compute pool `PYOMO_COMPUTE_POOL` created
- Tables `SALES_PREDICTIONS` and `ORDERS` populated with data


## 1. Setup and Imports


In [ ]:
# Import required libraries
from snowflake.snowpark.context import get_active_session
from snowflake.ml.jobs import submit_file
import time

# Get the active Snowflake session (automatically available in Snowflake notebooks)
session = get_active_session()
print(f"Connected to: {session.get_current_database()}.{session.get_current_schema()}")


## 2. Verify Prerequisites


In [ ]:
# Check compute pool status
compute_pool_name = "PYOMO_COMPUTE_POOL"

pool_status = session.sql(f"DESCRIBE COMPUTE POOL {compute_pool_name}").collect()
print(f"Compute Pool Status:")
for row in pool_status:
    print(f"  {row}")


In [ ]:
# Verify the optimization script is in the stage
stage_name = "ML_JOBS_STAGE"

files = session.sql(f"LIST @{stage_name}").collect()
print(f"Files in {stage_name}:")
for f in files:
    print(f"  {f['name']}")


In [ ]:
# Check data tables
sales_count = session.table("SALES_PREDICTIONS").count()
orders_count = session.table("ORDERS").count()

print(f"Sales Predictions: {sales_count} rows")
print(f"Orders: {orders_count} rows")


## 3. Preview Input Data


In [ ]:
# View demand summary by product
demand_summary = session.sql("""
    SELECT 
        product_id,
        product_name,
        SUM(predicted_sales) as total_demand,
        AVG(unit_price) as avg_price
    FROM SALES_PREDICTIONS
    GROUP BY product_id, product_name
    ORDER BY product_id
""").to_pandas()

print("Demand Summary by Product:")
demand_summary


In [ ]:
# View available supply by product
supply_summary = session.sql("""
    SELECT 
        product_id,
        COUNT(*) as num_suppliers,
        SUM(quantity_available) as total_available,
        ROUND(AVG(unit_cost), 2) as avg_cost,
        MIN(unit_cost) as min_cost,
        MAX(unit_cost) as max_cost
    FROM ORDERS
    GROUP BY product_id
    ORDER BY product_id
""").to_pandas()

print("Supply Summary by Product:")
supply_summary


## 4. Submit the ML Job


In [ ]:
# Configuration
COMPUTE_POOL = "PYOMO_COMPUTE_POOL"
STAGE_NAME = "ML_JOBS_STAGE"
SCRIPT_PATH = "@ML_JOBS_STAGE/INVENTORY_OPTIMIZATION.py"

print(f"Submitting ML Job...")
print(f"  Script: {SCRIPT_PATH}")
print(f"  Compute Pool: {COMPUTE_POOL}")


In [ ]:
# Submit the optimization job
# Note: highspy provides the HiGHS solver which is pip-installable
# make sure external access to PIP is available PYPI_ACCESS_INTEGRATION
job = submit_file(
    file_path=SCRIPT_PATH,
    compute_pool=COMPUTE_POOL,
    stage_name=STAGE_NAME,
    session=session,
    pip_requirements=[
        "pyomo",
        "highspy",  # HiGHS solver - pip installable MILP solver
        "pandas",
    ],
    external_access_integrations=["PYPI_ACCESS_INTEGRATION"]
)

print(f"✓ Job submitted successfully!")
print(f"  Job ID: {job.id}")
print(f"  Status: {job.status}")


## 5. Monitor Job Progress


In [ ]:
# Wait for job completion with progress updates
print(f"Waiting for job {job.id} to complete...")

while job.status not in ["COMPLETED", "FAILED", "CANCELLED"]:
    print(f"  Status: {job.status}")
    time.sleep(10)
    job.refresh()  # Refresh job status

print(f"\nFinal Status: {job.status}")


In [ ]:
# Get job logs
if job.status == "COMPLETED":
    print("Job Logs:")
    print("=" * 60)
    logs = job.get_logs()
    print(logs)
else:
    print(f"Job failed with status: {job.status}")
    try:
        print(job.get_logs())
    except:
        pass


## 6. View Optimization Results


In [ ]:
# Get the latest optimization run summary
summary = session.sql("""
    SELECT *
    FROM OPTIMIZATION_SUMMARY
    ORDER BY run_timestamp DESC
    LIMIT 1
""").to_pandas()

print("Latest Optimization Summary:")
summary.T


In [ ]:
# Get the run_id of the latest optimization
latest_run_id = summary['RUN_ID'].iloc[0]
print(f"Latest Run ID: {latest_run_id}")


In [ ]:
# View detailed order allocations
allocations = session.sql(f"""
    SELECT 
        order_id,
        product_id,
        supplier_name,
        allocated_quantity,
        unit_cost,
        total_cost,
        expected_delivery_date
    FROM OPTIMIZED_ORDER_ALLOCATION
    WHERE run_id = '{latest_run_id}'
    ORDER BY product_id, total_cost DESC
""").to_pandas()

print(f"Order Allocations for Run {latest_run_id}:")
allocations


In [ ]:
# Summary by product - compare demand vs allocation
comparison = session.sql(f"""
    WITH demand AS (
        SELECT product_id, SUM(predicted_sales) as total_demand
        FROM SALES_PREDICTIONS
        GROUP BY product_id
    ),
    allocation AS (
        SELECT product_id, 
               SUM(allocated_quantity) as total_allocated,
               SUM(total_cost) as total_cost
        FROM OPTIMIZED_ORDER_ALLOCATION
        WHERE run_id = '{latest_run_id}'
        GROUP BY product_id
    )
    SELECT 
        d.product_id,
        d.total_demand,
        a.total_allocated,
        ROUND((a.total_allocated / d.total_demand) * 100, 1) as coverage_pct,
        a.total_cost,
        ROUND(a.total_cost / a.total_allocated, 2) as avg_unit_cost
    FROM demand d
    JOIN allocation a ON d.product_id = a.product_id
    ORDER BY d.product_id
""").to_pandas()

print("Demand vs Allocation Comparison:")
comparison


## 7. Visualize Results


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Bar chart: Demand vs Allocation by Product
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(comparison))
width = 0.35

bars1 = ax.bar(x - width/2, comparison['TOTAL_DEMAND'], width, label='Demand', color='#2196F3')
bars2 = ax.bar(x + width/2, comparison['TOTAL_ALLOCATED'], width, label='Allocated', color='#4CAF50')

ax.set_xlabel('Product ID')
ax.set_ylabel('Quantity')
ax.set_title('Inventory Optimization: Demand vs Allocated Quantity')
ax.set_xticks(x)
ax.set_xticklabels(comparison['PRODUCT_ID'])
ax.legend()

# Add value labels
for bar in bars1:
    height = bar.get_height()
    ax.annotate(f'{int(height)}', xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0, 3), textcoords="offset points", ha='center', va='bottom', fontsize=8)

for bar in bars2:
    height = bar.get_height()
    ax.annotate(f'{int(height)}', xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0, 3), textcoords="offset points", ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()


In [ ]:
# Pie chart: Cost distribution by product
fig, ax = plt.subplots(figsize=(8, 8))

colors = ['#FF6384', '#36A2EB', '#FFCE56', '#4BC0C0', '#9966FF']
explode = [0.02] * len(comparison)

ax.pie(comparison['TOTAL_COST'], labels=comparison['PRODUCT_ID'], autopct='%1.1f%%',
       colors=colors, explode=explode, shadow=True, startangle=90)
ax.set_title('Cost Distribution by Product')

plt.tight_layout()
plt.show()
